This is part 2 of the web scraping project. Part 1 ended storing the national park trail dictionary so it could be opened by a separate Jupyter Notebook (this one) in the event the internet is lost. This document picks up at that point.

In [46]:
import os
from selenium import webdriver
import pandas as pd
import statsmodels.api as sm
import requests
import csv
import time
import seaborn as sns
from bs4 import BeautifulSoup as bs
import numpy as np
import matplotlib as plt

In [2]:
%store -r np_trail_dict
print(np_trail_dict)

{'yellowstone-national-park': ['https://new.hikingproject.com/trail/7008581/brink-of-lower-falls-trail', 'https://new.hikingproject.com/trail/7084138/grand-prismatic-overlook', 'https://new.hikingproject.com/trail/7008842/midway-geyser-basin-trail', 'https://new.hikingproject.com/trail/7009016/upper-geyser-basin-trail', 'https://new.hikingproject.com/trail/7008668/fountain-paint-pot-trail', 'https://new.hikingproject.com/trail/7008553/artist-point-overlook-trail', 'https://new.hikingproject.com/trail/7008920/red-rock-point-trail', 'https://new.hikingproject.com/trail/7008878/mud-volcano-trail', 'https://new.hikingproject.com/trail/7017141/osprey-falls-trail', 'https://new.hikingproject.com/trail/7009012/uncle-toms-trail', 'https://www.hikingproject.com/trail/7008885/north-rim-trail', 'https://www.hikingproject.com/trail/7008974/south-rim-trail', 'https://www.hikingproject.com/trail/7008805/mammoth-terraces-trail', 'https://www.hikingproject.com/trail/7008683/heart-lake-trail', 'https:/

Now from the dictionary of links, the next step is begin to create a new dictionary that will become our database. For this dictionary, the trail website will be the keyname. the values will be a dictionary for each trail. 

In [3]:
np_trails={}
test_trails={}

In [14]:
#To test getting the link from a dictionary, this dictionary was created. 
#Testing of the extraction code will be done on this first
testtrails={'yellowstone-national-park': ['https://www.hikingproject.com/trail/7008687/hoodoo-basin-trail']}

The next two cells are commented out. To avoid unnecessarily extra pings to the hikingproject.com trail sites, a request was made to one site and the HTML from that site was used with BeautifulSoup. Using the text in the beautiful soup, the extraction code could be tested, refined and perfected using the code in the second box. 

In [5]:
#This box was use to get a trail page HTML to test to extraction. 
#It is separate in the event the trail needed to change, and so the site was not parsed every time the below code was parsed.

#testurl='https://new.hikingproject.com/trail/7084138/grand-prismatic-overlook'
#response=requests.get(testtrails)
#html=response.text
#soup=bs(html)

In [6]:
#This box was use to test extracting the elements from the test site. 
#It was then copied and used below with the rest of the sites

#ratingpair=soup.find(class_='stat-box').text.split('\n    from\n    ')
#avg_rating=float(ratingpair[0].strip())
#num_raters=(ratingpair[1]).split(' votes')[0]
#datapoints=[]
#for span in soup.find(id='trail-stats-bar').find_all(class_='imperial'):
#    datapoints.append(span.text.strip('\n').strip())
#datapoints
#soup.find(class_='stat-block ml-2 mr-1 mt-1').find(class_='metric').next.next.next.next.next.next.next
#     .next.next.next.next.text
#grades=[]
#for grade in soup.find(class_='stat-block ml-1 mt-1').find_all('h3'):
 #   grades.append(int(grade.text.replace("%", "")))
#soup.find(id='more-checkins-button').text.strip().split(" ")[1]
#state=soup.find('ol').text.split('\n')[2]
#soup.find(class_='inline mr-2').text.strip()[5:])

In [15]:
#to reduce pings to the site, wait to run this. test on one site with testurl, but that is only one ping. 
for key, value in testtrails.items():
    for trail in value:
        #add a if statement here, passing if the website is in the final dictionary. skips pinging the same few pages.
        time.sleep(1)
        response=requests.get(trail)
        html=response.text
        soup=bs(html)
        difficulty=soup.find(class_='difficulty-text').text #1
        ratingpair=soup.find(class_='stat-box').text.split('\n    from\n    ') 
        avg_rating=float(ratingpair[0].strip()) #2
        num_raters=int((ratingpair[1]).split(' vote')[0]) #3
        datapoints=[]
        for span in soup.find(id='trail-stats-bar').find_all(class_='imperial'):
            datapoints.append(span.text.strip('\n').strip()) #4,5,6,7,8 (distance, high, low, up, down)
        trail_type=soup.find(class_='stat-block ml-2 mr-1 mt-1' #9
                            ).find(class_='metric').next.next.next.next.next.next.next.next.next.next.next.text
        grades=[]
        for grade in soup.find(class_='stat-block ml-1 mt-1').find_all('h3'):
            grades.append(float(grade.text.replace("%", ""))) #10, #11
        try:
            checkins=int(soup.find(id='more-checkins-button').text.strip().split(" ")[1]) #12
        except AttributeError:
            checkins=0
        state=soup.find('ol').text.split('\n')[2] #13
        dogs=soup.find(class_='inline mr-2').text.strip()[5:]
    
        
        test_trails[trail]={'park_name':key.replace('-','_'), 'website':trail,
                            'difficulty':difficulty, 'avg_rating':avg_rating,
                           'num_raters':num_raters, 'distance_(miles)':float(datapoints[0].split(' ')[0]), 
                           'high_(ft)': int(datapoints[1][:-1].replace(",","")), 'low_(ft)': int(datapoints[2][:-1].replace(",","")),
                           'up(ft)': int(datapoints[3][:-1].replace(",","")), 'down(ft)':int(datapoints[4][:-1].replace(",","")),
                           'trail_type': trail_type, 'average_grade(%)': grades[0], 'max_grade(%)':grades[1],
                           'checkins':checkins, 'State':state, 'dog_policy':dogs}
test_trails

{'https://www.hikingproject.com/trail/7008683/heart-lake-trail': {'park_name': 'yellowstone_national_park',
  'website': 'https://www.hikingproject.com/trail/7008683/heart-lake-trail',
  'difficulty': 'Intermediate',
  'avg_rating': 5.0,
  'num_raters': 4,
  'distance_(miles)': 19.4,
  'high_(ft)': 8141,
  'low_(ft)': 6965,
  'up(ft)': 858,
  'down(ft)': 1687,
  'trail_type': 'Point to Point',
  'average_grade(%)': 2.0,
  'max_grade(%)': 25.0,
  'checkins': 0,
  'State': 'WY',
  'dog_policy': 'No Dogs'},
 'https://www.hikingproject.com/trail/7008687/hoodoo-basin-trail': {'park_name': 'yellowstone_national_park',
  'website': 'https://www.hikingproject.com/trail/7008687/hoodoo-basin-trail',
  'difficulty': 'Difficult',
  'avg_rating': 5.0,
  'num_raters': 1,
  'distance_(miles)': 10.0,
  'high_(ft)': 10462,
  'low_(ft)': 7678,
  'up(ft)': 3616,
  'down(ft)': 843,
  'trail_type': 'Point to Point',
  'average_grade(%)': 8.0,
  'max_grade(%)': 33.0,
  'checkins': 0,
  'State': 'WY',
  'dog

In [8]:
#As can be seen above, it successfully makes a dictionary of the values extracted from the trail site. 
#Testing converting this to a dataframe:
trailstest_df=pd.DataFrame.from_dict(test_trails, orient='index').reset_index()
trailstest_df

,index,park_name,website,difficulty,avg_rating,num_raters,distance_(miles),high_(ft),low_(ft),up(ft),down(ft),trail_type,average_grade(%),max_grade(%),checkins,State,dog_policy
0,https://www.hikingproject.com/trail/7008683/he...,yellowstone_national_park,https://www.hikingproject.com/trail/7008683/he...,Intermediate,5.0,4,19.4,8141,6965,858,1687,Point to Point,2.0,25.0,0,WY,No Dogs


Now that the code successfully produces the database desired, it will be used on all the trail websites. The last step of the for loop is adding the trail to the dictionary. This is so in the event of a crash/loss of internet the trail will not be added to the dictionary. When the program is ran again, the 'if trail in np_trails.keys()' section of the code will cause the for loop to skip that trail, since the website has already been scraped. It will skip all the already scrapped sites, and pick up where it left off in the process.  

In [9]:
errortrails={}
np_trails={}

In [22]:
for key, value in np_trail_dict.items():
    for trail in value:
        if trail in np_trails.keys():
            continue
        else:
            time.sleep(1)
            response=requests.get(trail)
            html=response.text
            soup=bs(html)
            difficulty=soup.find(class_='difficulty-text').text #1
            ratingpair=soup.find(class_='stat-box').text.split('\n    from\n    ') 
            avg_rating=float(ratingpair[0].strip()) #2
            num_raters=int((ratingpair[1]).split(' vote')[0]) #3
            datapoints=[]
            for span in soup.find(id='trail-stats-bar').find_all(class_='imperial'):
                datapoints.append(span.text.strip('\n').strip()) #4,5,6,7,8 (distance, high, low, up, down)
            trail_type=soup.find(class_='stat-block ml-2 mr-1 mt-1' #9
                                ).find(class_='metric').next.next.next.next.next.next.next.next.next.next.next.text
            grades=[]
            for grade in soup.find(class_='stat-block ml-1 mt-1').find_all('h3'):
                grades.append(float(grade.text.replace("%", ""))) #10, #11
            try:
                checkins=int(soup.find(id='more-checkins-button').text.strip().split(" ")[1]) #12
            except AttributeError:
                checkins=0
            state=soup.find('ol').text.split('\n')[2] #13
            dogs=soup.find(class_='inline mr-2').text.strip()[5:]


            np_trails[trail]={'park_name':key.replace('-','_'), 'website':trail,
                                'difficulty':difficulty, 'avg_rating':avg_rating,
                               'num_raters':num_raters, 'distance_(miles)':float(datapoints[0].split(' ')[0]), 
                               'high_(ft)': int(datapoints[1][:-1].replace(",","")), 'low_(ft)': int(datapoints[2][:-1].replace(",","")),
                               'up(ft)': int(datapoints[3][:-1].replace(",","")), 'down(ft)':int(datapoints[4][:-1].replace(",","")),
                               'trail_type': trail_type, 'average_grade(%)': grades[0], 'max_grade(%)':grades[1],
                               'checkins':checkins, 'State':state, 'dog_policy':dogs}



In [23]:
len(np_trails.keys())

2938

In [21]:
np_trails.keys()

dict_keys(['https://new.hikingproject.com/trail/7008581/brink-of-lower-falls-trail', 'https://new.hikingproject.com/trail/7084138/grand-prismatic-overlook', 'https://new.hikingproject.com/trail/7008842/midway-geyser-basin-trail', 'https://new.hikingproject.com/trail/7009016/upper-geyser-basin-trail', 'https://new.hikingproject.com/trail/7008668/fountain-paint-pot-trail', 'https://new.hikingproject.com/trail/7008553/artist-point-overlook-trail', 'https://new.hikingproject.com/trail/7008920/red-rock-point-trail', 'https://new.hikingproject.com/trail/7008878/mud-volcano-trail', 'https://new.hikingproject.com/trail/7017141/osprey-falls-trail', 'https://new.hikingproject.com/trail/7009012/uncle-toms-trail', 'https://www.hikingproject.com/trail/7008885/north-rim-trail', 'https://www.hikingproject.com/trail/7008974/south-rim-trail', 'https://www.hikingproject.com/trail/7008805/mammoth-terraces-trail', 'https://www.hikingproject.com/trail/7008683/heart-lake-trail', 'https://www.hikingproject.c

In [30]:
np_trail_df=pd.DataFrame.from_dict(np_trails, orient='index').reset_index()


In [31]:
np_trail_df.head()

,index,park_name,website,difficulty,avg_rating,num_raters,distance_(miles),high_(ft),low_(ft),up(ft),down(ft),trail_type,average_grade(%),max_grade(%),checkins,State,dog_policy
0,https://new.hikingproject.com/trail/7008581/br...,yellowstone_national_park,https://new.hikingproject.com/trail/7008581/br...,Intermediate/Difficult,4.7,48,0.4,7720,7473,0,248,Point to Point,13.0,40.0,103,WY,No Dogs
1,https://new.hikingproject.com/trail/7084138/gr...,yellowstone_national_park,https://new.hikingproject.com/trail/7084138/gr...,Easy/Intermediate,4.8,35,0.3,7357,7278,79,69,Point to Point,9.0,11.0,61,WY,No Dogs
2,https://new.hikingproject.com/trail/7008842/mi...,yellowstone_national_park,https://new.hikingproject.com/trail/7008842/mi...,Easy,4.6,32,0.5,7267,7226,41,11,Point to Point,2.0,5.0,102,WY,No Dogs
3,https://new.hikingproject.com/trail/7009016/up...,yellowstone_national_park,https://new.hikingproject.com/trail/7009016/up...,Easy,4.4,33,2.1,7369,7321,81,80,Point to Point,1.0,5.0,101,WY,No Dogs
4,https://new.hikingproject.com/trail/7008668/fo...,yellowstone_national_park,https://new.hikingproject.com/trail/7008668/fo...,Easy,4.6,28,0.6,7315,7288,35,27,Point to Point,2.0,6.0,107,WY,No Dogs


In [35]:
np_trail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             2938 non-null   object 
 1   park_name         2938 non-null   object 
 2   website           2938 non-null   object 
 3   difficulty        2938 non-null   object 
 4   avg_rating        2938 non-null   float64
 5   num_raters        2938 non-null   int64  
 6   distance_(miles)  2938 non-null   float64
 7   high_(ft)         2938 non-null   int64  
 8   low_(ft)          2938 non-null   int64  
 9   up(ft)            2938 non-null   int64  
 10  down(ft)          2938 non-null   int64  
 11  trail_type        2938 non-null   object 
 12  average_grade(%)  2938 non-null   float64
 13  max_grade(%)      2938 non-null   float64
 14  checkins          2938 non-null   int64  
 15  State             2938 non-null   object 
 16  dog_policy        2938 non-null   object 


In [41]:
np_trail_df.num_raters.value_counts()

1      704
2      466
0      414
3      316
4      230
      ... 
78       1
36       1
84       1
57       1
110      1
Name: num_raters, Length: 67, dtype: int64

In [50]:
#save the file so can open in new Jupyter notebook.
np_trail_df.to_excel('National_Park_trail_database.xlsx')

In [ ]:
#this project is continued in part 3